In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import dice_ml
from dice_ml.utils import helpers # helper functions
from dice_ml import Data,Model,Dice
import numpy as np
from xgboost import XGBClassifier


In [2]:
dataframe_heart_disease = pd.read_csv("heart_statlog_cleveland_hungary_final.csv")

dataframe_heart_disease =  dataframe_heart_disease.dropna()
dataframe_heart_disease = dataframe_heart_disease[dataframe_heart_disease['chol'] !=0]

In [3]:
dataframe_heart_disease.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0
5,39,1,3,120,339,0,0,170,0,0.0,1,0
6,45,0,2,130,237,0,0,170,0,0.0,1,0
7,54,1,2,110,208,0,0,142,0,0.0,1,0
8,37,1,4,140,207,0,0,130,1,1.5,2,1
9,48,0,2,120,284,0,0,120,0,0.0,1,0


In [4]:
# Split the data into features and target label
y = dataframe_heart_disease.target
X = dataframe_heart_disease.drop(['target'], axis=1)

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
numerical = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical = X_train.columns.difference(numerical)

In [7]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

In [8]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', XGBClassifier())])
xgb_model = clf.fit(X_train, y_train)


In [9]:
param_grid = {
    'classifier__max_depth': [3, 4, 5],
    'classifier__learning_rate': [0.01, 0.1, 0.5],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__gamma': [0, 0.1, 0.5]
}


In [10]:
# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(xgb_model, param_grid=param_grid)
grid_search.fit(X_train, y_train)


# Print the best parameters and the score on the test set
print("Best parameters: ", grid_search.best_params_)
print("Test set score: ", grid_search.score(X_test, y_test))

Best parameters:  {'classifier__gamma': 0, 'classifier__learning_rate': 0.5, 'classifier__max_depth': 4, 'classifier__n_estimators': 300}
Test set score:  0.9166666666666666


In [11]:
# create the pipeline with the XGBClassifier
pipeline = Pipeline(steps=[
    ('preprocessor', transformations),
    ('classifier', XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=300, gamma=0))
])


In [12]:
# train the model on the entire training set
xgb_pipeline = pipeline.fit(X_train, y_train)

# make predictions on the test set
y_pred = xgb_pipeline.predict(X_test)


In [13]:
train_data = pd.concat([X_train, y_train], axis=1)


# Create a DICE data object
d = Data(dataframe=pd.DataFrame(train_data, columns=dataframe_heart_disease.columns), continuous_features=['age', 'trestbps', 'chol', 'thalach', 'oldpeak'],outcome_name='target')

# Create a DICE model object
m = Model(model=xgb_pipeline, backend="sklearn")


# Define a test instance to explain

test_instance = X_test[10:11]



In [14]:
# Function to calculate the upper bound for feature2
def calculate_upper_bound(x):
    if feature1_value is not None:
        return 220 - x['age']
    else:
        return float('inf')

feature_constraint= {
    'chol':[100,200],
    'thalach': [100, calculate_upper_bound]

}

In [46]:
# Create a DICE explainer object and generate an explanation
exp = Dice(d, m, method='genetic').generate_counterfactuals(test_instance, total_CFs=20, desired_class="opposite", 
                                                            features_to_vary=["trestbps", "chol", "thalach"],
                                                            proximity_weight=1,diversity_weight=1.5)
 
# Print the explanation
print(exp.visualize_as_dataframe())


100%|██████████| 1/1 [00:29<00:00, 29.65s/it]

Query instance (original outcome : 1)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,target
0,56.0,1,4,130.0,283.0,1,2,103.0,1,1.6,3,1



Diverse Counterfactual set (new outcome: 0)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,target
0,56.0,1,4,130.0,209.0,1,2,127.0,1,1.6,3,0
0,56.0,1,4,130.0,215.0,1,2,135.0,1,1.6,3,0
0,56.0,1,4,130.0,215.0,1,2,138.0,1,1.6,3,0
0,56.0,1,4,130.0,202.0,1,2,137.0,1,1.6,3,0
0,56.0,1,4,130.0,213.0,1,2,140.0,1,1.6,3,0
0,56.0,1,4,130.0,236.0,1,2,144.0,1,1.6,3,0
0,56.0,1,4,130.0,231.0,1,2,146.0,1,1.6,3,0
0,56.0,1,4,130.0,245.0,1,2,150.0,1,1.6,3,0
0,56.0,1,4,130.0,224.0,1,2,150.0,1,1.6,3,0
0,56.0,1,4,130.0,207.0,1,2,153.0,1,1.6,3,0


None


For subjects with cholestestoral in range … and have target=0, their target would be 1 if cholesterol were greater than …
For subjects with cholesterol in range … and thalac in range … and target = 0, their target would be 1 if cholesterol were in range …. thalac were in range …
Monotonicity
Shape of the Curve between variables 
Cholestrol can either increase the chance or it has no effect
(Constrained XgBoost, Normalizing flows,Linear Models)

In [47]:
imp = Dice(d, m, method='genetic').local_feature_importance(test_instance, cf_examples_list=exp.cf_examples_list)
print(imp.local_importance)

[{'chol': 1.0, 'thalach': 1.0, 'trestbps': 0.3157894736842105, 'sex': 0.0, 'cp': 0.0, 'fbs': 0.0, 'restecg': 0.0, 'exang': 0.0, 'slope': 0.0, 'age': 0.0, 'oldpeak': 0.0}]


In [23]:
cfs_dataframe =exp